In [1]:
import os
import openai

In [2]:
os.environ["OPENAI_API_KEY"] = "sk-your api"
openai.api_key = os.environ["OPENAI_API_KEY"]

In [3]:
from IPython.display import Markdown, display

In [4]:
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
    select,
)


In [5]:
engine = create_engine("sqlite:///:memory:")
metadata_obj = MetaData()

In [6]:
# create city SQL table
table_name = "city_stats"
city_stats_table = Table(
    table_name,
    metadata_obj,
    Column("city_name", String(16), primary_key=True),
    Column("population", Integer),
    Column("country", String(16), nullable=False),
)
metadata_obj.create_all(engine)

In [7]:
from llama_index import SQLDatabase, ServiceContext
from llama_index.llms import OpenAI

In [8]:
llm = OpenAI(temperature=0.1, model="gpt-3.5-turbo")
service_context = ServiceContext.from_defaults(llm=llm)

In [9]:
sql_database = SQLDatabase(engine, include_tables=["city_stats"])

In [10]:
sql_database = SQLDatabase(engine, include_tables=["city_stats"])
from sqlalchemy import insert

rows = [
    {"city_name": "Toronto", "population": 2930000, "country": "Canada"},
    {"city_name": "Tokyo", "population": 13960000, "country": "Japan"},
    {"city_name": "Chicago","population": 2679000,"country": "United States",},
    {"city_name": "Seoul", "population": 9776000, "country": "South Korea"},
    {"city_name": "Abuja", "population": 8776000, "country": "Nigeria"}
]
for row in rows:
    stmt = insert(city_stats_table).values(**row)
    with engine.begin() as connection:
        cursor = connection.execute(stmt)

In [11]:
stmt = select(
    city_stats_table.c.city_name,
    city_stats_table.c.population,
    city_stats_table.c.country,
).select_from(city_stats_table)

with engine.connect() as connection:
    results = connection.execute(stmt).fetchall()
    print(results)

[('Toronto', 2930000, 'Canada'), ('Tokyo', 13960000, 'Japan'), ('Chicago', 2679000, 'United States'), ('Seoul', 9776000, 'South Korea'), ('Abuja', 8776000, 'Nigeria')]


In [12]:
from sqlalchemy import text

with engine.connect() as con:
    rows = con.execute(text("SELECT city_name from city_stats"))
    for row in rows:
        print(row)

('Abuja',)
('Chicago',)
('Seoul',)
('Tokyo',)
('Toronto',)


In [13]:
from sqlalchemy import text

with engine.connect() as con:
    rows = con.execute(text("SELECT city_name from city_stats where population = 8776000"))
    for row in rows:
        print(row)

('Abuja',)


In [14]:
from llama_index.indices.struct_store.sql_query import NLSQLTableQueryEngine

query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database,
    tables=["city_stats"],
)


In [15]:
query_str = "Which city has the highest population?"
response = query_engine.query(query_str)

In [16]:
response

Response(response='The city with the highest population is Tokyo.', source_nodes=[NodeWithScore(node=TextNode(id_='9e118422-a378-4767-92de-b17720326498', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text="[('Tokyo',)]", start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=None)], metadata={'9e118422-a378-4767-92de-b17720326498': {}, 'sql_query': 'SELECT city_name FROM city_stats ORDER BY population DESC LIMIT 1;', 'result': [('Tokyo',)], 'col_keys': ['city_name']})

In [19]:
response = query_engine.query("What is the population of the city with highest population")

In [23]:
response

Response(response='The population of the city with the highest population is 13,960,000.', source_nodes=[NodeWithScore(node=TextNode(id_='724b6543-e5f9-436a-a1a1-255f1d7f149b', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='c292764db740ed4d521892eb71017fb572fea01f5919645774986cb6b013d47b', text='[(13960000,)]', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=None)], metadata={'724b6543-e5f9-436a-a1a1-255f1d7f149b': {}, 'sql_query': 'SELECT population\nFROM city_stats\nORDER BY population DESC\nLIMIT 1;', 'result': [(13960000,)], 'col_keys': ['population']})

In [17]:
response = query_engine.query("Return the top 3Return the top 5 cities (along with their populations) with the highest population cities (along with their populations) with the highest population")

In [18]:
response

Response(response='The top 5 cities with the highest population are Tokyo with a population of 13,960,000, Seoul with a population of 9,776,000, Abuja with a population of 8,776,000, Toronto with a population of 2,930,000, and Chicago with a population of 2,679,000.', source_nodes=[NodeWithScore(node=TextNode(id_='54dad1f5-b02a-427e-8bbc-709f8f584ccb', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text="[('Tokyo', 13960000), ('Seoul', 9776000), ('Abuja', 8776000), ('Toronto', 2930000), ('Chicago', 2679000)]", start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=None)], metadata={'54dad1f5-b02a-427e-8bbc-709f8f584ccb': {}, 'sql_query': 'SELECT city_name, population\nFROM city_stats\nORDER BY population DESC\nLIMIT 5;', 'result': [('Tokyo', 13960000), ('Seoul', 9776000), ('Abuja', 8776000), ('Toronto', 2930000), ('Chicago', 2

In [43]:
# create city SQL table
table_name = "state1"
state_table = Table(
    table_name,
    metadata_obj,
    Column("State", String(16), primary_key=True),
    Column("population", Integer),
    Column("Capital", String(16), nullable=False),
)
metadata_obj.create_all(engine)

In [44]:
sql_database = SQLDatabase(engine, include_tables=["state1","city_stats"])

In [45]:

from sqlalchemy import insert

rows = [
    {"State": "Jigawa", "population": 2930000, "Capital": "Dutse"},
    {"State": "Borno", "population": 13960000, "Capital": "Maiduguri"},
    {"State": "Yobe","population": 2679000,"Capital": "Damaturu",},
    {"State": "Zamfara", "population": 9776000, "Capital": "Gusau"},
    {"State": "Lagos", "population": 8776000, "Capital": "Ikeja"}
]
for row in rows:
    stmt = insert(state_table).values(**row)
    with engine.begin() as connection:
        cursor = connection.execute(stmt)

In [47]:
from sqlalchemy import text

with engine.connect() as con:
    rows = con.execute(text("SELECT State from state1"))
    for row in rows:
        print(row)

('Borno',)
('Jigawa',)
('Lagos',)
('Yobe',)
('Zamfara',)


In [38]:
from llama_index.indices.struct_store.sql_query import (
    SQLTableRetrieverQueryEngine,
)
from llama_index.objects import (
    SQLTableNodeMapping,
    ObjectIndex,
    SQLTableSchema,
)
from llama_index import VectorStoreIndex

In [50]:
table_node_mapping = SQLTableNodeMapping(sql_database)
table_schema_objs = [
    (SQLTableSchema(table_name="city_stats")),
    (SQLTableSchema(table_name="state1"))
]

In [51]:
obj_index = ObjectIndex.from_objects(
    table_schema_objs,
    table_node_mapping,
    VectorStoreIndex,
)

In [52]:
query_engine = SQLTableRetrieverQueryEngine(
    sql_database, obj_index.as_retriever(similarity_top_k=1)
)

In [57]:
response = query_engine.query("Which city has the highest population?")
display(Markdown(f"<b>{response}</b>"))

<b>The city with the highest population is Tokyo.</b>

In [58]:
from llama_index.retrievers import NLSQLRetriever

# default retrieval (return_raw=True)
nl_sql_retriever = NLSQLRetriever(
    sql_database, tables=["city_stats","state1"], return_raw=True
)

In [59]:
results = nl_sql_retriever.retrieve(
    "Return the top 5 cities (along with their populations) with the highest population."
)

In [60]:
display(Markdown(f"<b>{results}</b>"))

<b>[NodeWithScore(node=TextNode(id_='132a2973-b9dd-46b0-93d1-a7a0a7a71ace', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='9ebeaebf21e2debc09c9e47ef142ecfd7128f292d2e9c99b2e1598d17c013229', text="[('Tokyo', 13960000), ('Seoul', 9776000), ('Abuja', 8776000), ('Toronto', 2930000), ('Chicago', 2679000)]", start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=None)]</b>